# Library

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import pandas as pd
import os
import time
import numpy as np
import re

# Setup Selenium

In [2]:
#define chromdriver option, then create webdriver object
options=webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome("chromedriver.exe",options=options)

C:\Users\arif\AppData\Local\Temp\ipykernel_9192\2485549817.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe",options=options)


# Scrapper + Cleansing Functs

In [43]:
#function to scroll down the page
def scroll_down():
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.send_keys(Keys.PAGE_DOWN)
    
#function to generate list by its name in df_columns list
def list_generator():
    
    #list variable name
    global df_columns
    df_columns=['hero','pick_total','pick_W','pick_L','pick_WR','pick_percent_T','blue_s_total','blue_s_W','blue_s_L','blue_s_WR','red_s_total',\
                'red_s_W','red_s_L','red_s_WR','ban_total','ban_percent_T','pick_ban_total','pick_ban_percent_T','played_by','played_with','played_vss']
    
    #loop to covert it to list
    for i in df_columns :
        globals()[i] = list()
    return
list_generator()

def extract_common() :
   
    #get list to hold data
    ls_common_var=[hero,pick_total,pick_W,pick_L,pick_WR,pick_percent_T,blue_s_total,blue_s_W,blue_s_L,blue_s_WR,\
     red_s_total,red_s_W,red_s_L,red_s_WR,ban_total,ban_percent_T,pick_ban_total,pick_ban_percent_T]
    
    #get all data from class dota-stat-row
    extracted_value=driver.find_elements(By.CLASS_NAME,'dota-stat-row')
    
    #loop to extract data
    for data in extracted_value :

        #parse text
        list_value=data.text.split('\nhide')[0].split(' ')[2:]
        
        #append it to list
        if len(list_value)==len(ls_common_var) :
            for i,j in zip(ls_common_var,list_value) :
                i.append(j)
        else :
            list_value.pop(0)
            for i,j in zip(ls_common_var,list_value) :
                i.append(j)

    #build df
    df=pd.DataFrame(ls_common_var).transpose()
    df.columns=list(df_columns[0:-3])
    df.drop_duplicates(inplace=True)
    
    #replace certain wrong hero names
    df=df.replace({"hero":{'Yi':'Luo Yi','Zhong':'Yu Zhong','Sun-Shin':'Yi Sun-Shin'}})
    return df

def show_hiden_stat():
    #clck show button
    for j in driver.find_elements(By.CSS_SELECTOR,'.mw-collapsible-toggle'):
        if j.text=='show' :
            while j.text=='show' :
                time.sleep(0.5)
                try :
                    j.click()
                except:
                    scroll_down()
                    j.click()

######--- Function to get stat detial
def extract_detail_name(elem):
    result=[]
    for i in elem.find_elements(By.XPATH, '*') :
        val=i.find_elements(By.TAG_NAME,'a')[0].get_attribute('href')
        result.append(val.split('/')[-1])
    #team=elem.find_elements(By.XPATH, '*')[0]
    #team=team.find_elements(By.TAG_NAME,'a')[0].get_attribute('href')
    return result

def exctract_detail_play(elem,category) :
    if category =='team' :
        elem=elem.find_elements(By.CLASS_NAME,"mw-collapsible-content")[0]
        val=extract_detail_name(elem)
    elif category =='teammate' :
        elem=elem.find_elements(By.CLASS_NAME,"mw-collapsible-content")[1]
        val=extract_detail_name(elem)
    elif category == 'enemy' :
        elem=elem.find_elements(By.CLASS_NAME,"mw-collapsible-content")[2]
        val=extract_detail_name(elem)
    return val
        
def get_detail_play() :
    
    #dict heros
    dict_heroes={}
    ls_heroes=[i.find_elements(By.TAG_NAME,'a')[1].text for i in driver.find_elements(By.CLASS_NAME,'dota-stat-row')]
    for i in ls_heroes :
        dict_heroes[i]={}
        dict_heroes[i]['team_that_used_hero']=[]
        dict_heroes[i]['teammate_heroes']=[]
        dict_heroes[i]['enemy_heroes']=[]
        dict_heroes[i]['played_by_team_stat']=[]
        dict_heroes[i]['played_with_heroes_stat']=[]
        dict_heroes[i]['played_against_heroes_stat']=[]
    dict_heroes

    #parent (table)
    parent=driver.find_elements(By.CLASS_NAME,'dota-stat-row')
        
    #extract value
    for i in parent:
        ls_team=exctract_detail_play(i,'team')
        ls_teammate_heroes=exctract_detail_play(i,'teammate')
        ls_enemy_heroes=exctract_detail_play(i,'enemy')
        heroname=i.find_elements(By.TAG_NAME,'a')[1].text
        
        elem_hero_team_stat=i.find_elements(By.CLASS_NAME,"mw-collapsible-content")[0] #hero that used by team stat
        ls_hero_team_stat=[i.text for i in elem_hero_team_stat.find_elements(By.XPATH, '*')]

        elem_hero_played_with_stat=i.find_elements(By.CLASS_NAME,"mw-collapsible-content")[1] #hero that used by team stat        
        ls_hero_played_with_stat=[i.text for i in elem_hero_played_with_stat.find_elements(By.XPATH, '*')] #hero that played with stat

        elem_hero_played_against_stat=i.find_elements(By.CLASS_NAME,"mw-collapsible-content")[2] #hero that used by team stat        
        ls_hero_played_against_stat=[i.text for i in elem_hero_played_against_stat.find_elements(By.XPATH, '*')] #hero that played with stat

        
        #print(ls_team)
        for j,k,l,m,n,o in zip(ls_team,ls_teammate_heroes,ls_enemy_heroes,ls_hero_team_stat,ls_hero_played_with_stat,ls_hero_played_against_stat) :
            dict_heroes[heroname]['team_that_used_hero'].append(j)
            dict_heroes[heroname]['teammate_heroes'].append(k)
            dict_heroes[heroname]['enemy_heroes'].append(l)
            dict_heroes[heroname]['played_by_team_stat'].append(m)
            dict_heroes[heroname]['played_with_heroes_stat'].append(n)
            dict_heroes[heroname]['played_against_heroes_stat'].append(o)
    
    return dict_heroes

######--- transformation function
def transform_to_df(dict_) :
    #build dataframe for data statistics
    df_stats=pd.DataFrame(columns=['hero','team_that_used_hero', 'teammate_heroes', 'enemy_heroes','played_by_team_stat',\
                    'played_with_heroes_stat','played_against_heroes_stat'])
    for i in dict_.keys() :
        data_current=pd.DataFrame.from_dict(dict_[i])
        data_current['hero']=i
        df_stats=pd.concat([df_stats,data_current[df_stats.columns]])
    df_stats=df_stats.reset_index().drop('index',axis=1)
    return df_stats

def replace_char(x) :
    ls_char=['-','(',')']
    for i in ls_char :
        if i == '-':
            x=x.replace('-',' ')
        else :
            x=x.replace(i,'')
    return x

def cleanse(df) :
    
    df=df.replace('-',0)
    
    df['pick_WR']=round(df['pick_WR'].str.replace('%','').astype(float)/100,2)
    df['pick_percent_T']=round(df['pick_percent_T'].str.replace('%','').astype(float)/100,2)
    df['blue_s_WR']=round(df['blue_s_WR'].str.replace('%','').astype(float)/100,2)
    df['red_s_WR']=round(df['red_s_WR'].str.replace('%','').astype(float)/100,2)
    df['ban_percent_T']=round(df['ban_percent_T'].str.replace('%','').astype(float)/100,2)
    df['pick_ban_percent_T']=round(df['pick_ban_percent_T'].str.replace('%','').astype(float)/100,2)
    
    df['played_total']=df['played_by_team_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[0]
    df['played_total_win']=df['played_by_team_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[1]
    df['played_total_lose']=df['played_by_team_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[2]

    df['ally_hero_played_total']=df['played_with_heroes_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[0]
    df['ally_hero_played_total_win']=df['played_with_heroes_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[1]
    df['ally_hero_played_total_lose']=df['played_with_heroes_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[2]

    df['enemy_hero_played_total']=df['played_against_heroes_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[0]
    df['enemy_hero_played_total_win']=df['played_against_heroes_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[1]
    df['enemy_hero_played_total_lose']=df['played_against_heroes_stat'].apply(lambda x: replace_char(x)).str.split(' ',expand=True)[2]

    df=df.fillna(0)
    return df

def scrape_icon() :
    ls_result=[]
    for hero in driver.find_elements(By.CLASS_NAME,'dota-stat-row') :
        icon_link=hero.find_element(By.TAG_NAME,'img').get_attribute('src')
        ls_result.append(icon_link)
    return ls_result

# Scraping Process

In [22]:
#get to the page
driver.get('https://liquipedia.net/mobilelegends/M5_World_Championship/Statistics')

#expan hiden stat
show_hiden_stat()

In [32]:
#scrape common value
df_common=extract_common()
df_common['hero_icon_link']=scrape_icon()
df_common.head(5)

,hero,pick_total,pick_W,pick_L,pick_WR,pick_percent_T,blue_s_total,blue_s_W,blue_s_L,blue_s_WR,red_s_total,red_s_W,red_s_L,red_s_WR,ban_total,ban_percent_T,pick_ban_total,pick_ban_percent_T,hero_icon_link
0,Valentina,85,41,44,48.24%,55.92%,41,19,22,46.34%,44,22,22,50.00%,32,21.05%,117,76.97%,https://liquipedia.net/commons/images/thumb/c/...
1,Bruno,84,47,37,55.95%,55.26%,41,22,19,53.66%,43,25,18,58.14%,51,33.55%,135,88.82%,https://liquipedia.net/commons/images/thumb/5/...
2,Terizla,79,42,37,53.16%,51.97%,44,25,19,56.82%,35,17,18,48.57%,33,21.71%,112,73.68%,https://liquipedia.net/commons/images/thumb/2/...
3,Faramis,67,34,33,50.75%,44.08%,24,10,14,41.67%,43,24,19,55.81%,64,42.11%,131,86.18%,https://liquipedia.net/commons/images/thumb/b/...
4,Arlott,66,29,37,43.94%,43.42%,26,8,18,30.77%,40,21,19,52.50%,54,35.53%,120,78.95%,https://liquipedia.net/commons/images/thumb/a/...


In [44]:
#scrape data detail
data_statistics=get_detail_play()

#transform to df
df_stats=transform_to_df(data_statistics)
df_stats.head(5)

,hero,team_that_used_hero,teammate_heroes,enemy_heroes,played_by_team_stat,played_with_heroes_stat,played_against_heroes_stat
0,Valentina,Blacklist_International,Terizla,Faramis,12 (8-4),27 (13-14),28 (13-15)
1,Valentina,Team_Lilgun,Bruno,Bruno,8 (5-3),22 (12-10),24 (12-12)
2,Valentina,Team_SMG,Claude,Lylia,8 (3-5),16 (8-8),22 (11-11)
3,Valentina,Deus_Vult,Akai,Arlott,8 (2-6),15 (6-9),20 (11-9)
4,Valentina,ONIC_Esports,Martis,Terizla,7 (5-2),15 (6-9),20 (11-9)


In [45]:
#merge all data
df_result=df_common.merge(df_stats,on='hero',how='left')

In [46]:
#clean 
df_clean=cleanse(df_result)
df_clean.head(5)

,hero,pick_total,pick_W,pick_L,pick_WR,pick_percent_T,blue_s_total,blue_s_W,blue_s_L,blue_s_WR,...,played_against_heroes_stat,played_total,played_total_win,played_total_lose,ally_hero_played_total,ally_hero_played_total_win,ally_hero_played_total_lose,enemy_hero_played_total,enemy_hero_played_total_win,enemy_hero_played_total_lose
0,Valentina,85,41,44,0.48,0.56,41,19,22,0.46,...,28 (13-15),12,8,4,27,13,14,28,13,15
1,Valentina,85,41,44,0.48,0.56,41,19,22,0.46,...,24 (12-12),8,5,3,22,12,10,24,12,12
2,Valentina,85,41,44,0.48,0.56,41,19,22,0.46,...,22 (11-11),8,3,5,16,8,8,22,11,11
3,Valentina,85,41,44,0.48,0.56,41,19,22,0.46,...,20 (11-9),8,2,6,15,6,9,20,11,9
4,Valentina,85,41,44,0.48,0.56,41,19,22,0.46,...,20 (11-9),7,5,2,15,6,9,20,11,9


In [48]:
#export to csv
df_clean.to_csv('dataset_pick_ban_m5.csv',index=False)